# Keras binary classification - IMDB Movie reviews

<table class="tfo-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/tensorflow/models/blob/master/samples/core/tutorials/keras/basic_text_classification.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /><span>Run in Google Colab</span></a>  
</td><td>
<a target="_blank"  href="https://github.com/tensorflow/models/blob/master/samples/core/tutorials/keras/basic_text_classification.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /><span>View source on GitHub</span></a></td></table>

##### Copyright 2018 The TensorFlow Authors; https://www.apache.org/licenses/LICENSE-2.0, MIT License


This notebook **classifies movie reviews as *positive* or *negative* ** using the text of the review. This is an example of binary—or two-class—classification, an important and widely applicable kind of machine learning problem. 

We'll use the [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) that contains the text of 50,000 movie reviews from the [Internet Movie Database](https://www.imdb.com/). These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are *balanced*, meaning they contain an equal number of positive and negative reviews. 

This notebook uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow.

In [ ]:
import tensorflow as tf
from tensorflow import keras

% matplotlib inline

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

from mymods.lauthom import *

import sys
sys.path.append('./pycode')

print(tf.__version__)

## Download the IMDB dataset

The IMDB dataset comes packaged with TensorFlow. It has already been preprocessed such that the reviews (sequences of words) have been converted to sequences of integers, where each integer represents a specific word in a dictionary.

The following code downloads the IMDB dataset to your machine (or uses a cached copy if you've already downloaded it):

In [ ]:
imdb = keras.datasets.imdb

vocab_size = 10000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)

The argument `num_words=10000` keeps the top 10,000 most frequently occurring words in the training data. The rare words are discarded to keep the size of the data managable.

## Explore the data 

Let's take a moment to understand the format of the data. The dataset comes preprocessed: each example is an array of integers representing the words of the movie review. Each label is an integer value of either 0 or 1, where 0 is a negative review and 1 is a positive review.

In [ ]:
print("Training entries(reviews): {}, labels(sentiment): {}".format(len(train_data), len(train_labels)))

The text of reviews have been converted to integers, where **each integer represents a specific word in a dictionary**. Here's what the first review looks like:

In [ ]:
print(train_data[0])

Movie reviews may be different lengths. The below code shows the **number of words in the first and second review**. Since inputs to a neural network must be the same length, we'll need to resolve this later.

In [ ]:
len(train_data[0]), len(train_data[1])

### Convert the integers back to words

It may be useful to know how to convert integers back to text. Here, we'll create a helper function to query a dictionary object that contains the integer to string mapping:

In [ ]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# Head of word dictionary
list(word_index.items())[:10]

In [ ]:
# Shift index and add 4 'placeholders'
word_index = {word:(index+3) for word, index in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

In [ ]:
# Create the reverse dictionary
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
def decode_review(text):
    """Decode review from list of integers to text (lowercase without symbols)"""
    return ' '.join([reverse_word_index.get(integer, '?') for integer in text])

Now we can use the `decode_review` function to display the text for the first review:

In [ ]:
decode_review(train_data[0])

## Prepare the data

The **reviews—the arrays of integers—must be converted to tensors** before fed into the neural network. This conversion can be done a couple ways:

* **Sparse vector**  
**One-hot-encode** the arrays to convert them into vectors of 0s and 1s. For example, the sequence  [3, 5] would become a 10,000-dimensional vector that is all zeros except for indices 3 and 5, which are ones. Then, make this the first layer in our network—a Dense layer—that can handle floating point vector data. This approach is memory intensive, though, requiring a **`num_reviews * num_words * 10,000-dimensional vector`** size matrix.

* **Dense vector**  
Alternatively, we can pad the arrays so they all have the same length, then create an integer tensor of shape **`num_reviews * (num_words + padding) * n-dimensional embedding vector`**. We can use an **embedding layer** transforming each integer to a n-dimensional 'word space' vector. Words are projected in a n-dimensional space. Alike words have similar vector values.

In this tutorial, we willl use the second approach. 

#### Padding
Since the movie reviews must be the same length, we will use the [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) function to standardize the lengths:

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

Let's look at the length of the examples now:

In [ ]:
len(train_data[0]), len(train_data[1])

And inspect the (now padded) first review:

In [ ]:
print(train_data[0])

## Build the model

The neural network is created by stacking layers—this requires two main architectural decisions:

* How many layers to use in the model?
* How many *hidden units* to use for each layer?

In this example, the input data consists of array of word-indices. The labels to predict are either 0 or 1. Let's build a model for this problem:

In [ ]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

The layers are stacked sequentially to build the classifier:

1. ** `Embedding` ** layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: `(batch, sequence, embedding)`.
2. ** `GlobalAveragePooling1D` ** layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input of variable length, in the simplest way possible.
3. ** `Dense` ** fully-connected layer with 16 hidden units.
4. ** `Dense` ** fully-connected layer with a single output node. Using the `sigmoid` activation function, this value is a float between 0 and 1, representing a probabilty, or confidence level.

### Hidden units

The above model has two intermediate or "hidden" layers, between the input and output. The number of outputs (units, nodes , or neurons) is the dimension of the representational space for the layer. In other words, the amount of freedom the network is allowed when learning an internal representation.

If a model has more hidden units (a higher-dimensional representation space), and/or more layers, then the network can learn more complex representations. However, it makes the network more computationally expensive and may lead to learning unwanted patterns—patterns that improve performance on training data but not on the test data. This is called *overfitting*, and we'll explore it later.

### Loss function and optimizer

A model need a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs of a probability (a single-unit layer with a sigmoid activation), we'll use the `binary_crossentropy` loss function. 

This isn't the only choice of loss function, you could, for instance, choose `mean_squared_error`. But, generally, `binary_crossentropy` is better for dealing with out probabilities—it measures the "distance" between probability distributions, or in our case, between the ground-truth distribution and the predictions.

Later, when we are exploring regression problems (say, to predict the price of a house), we will see how to use another loss function called mean squared error.

Now, configure the model to use an optimizer and a loss function:

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Create a validation set

When training, we want to check the accuracy of the model on data it hasn't seen before. Create a *validation set* by setting apart 10,000 examples from the original training data. (Why not use the testing set now? Our goal is to develop and tune our model using only the training data, then use the test data just once to evaluate our accuracy).

In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## Train the model

### Automatically stop training when the validation score doesn't improve

Train the model for N epochs in mini-batches of 512 samples and stop automatically when the validation loss does not improve significantly. 

We'll use a *callback* that tests a training condition for every epoch. If a set amount of epochs elapses without showing improvement, the training automatically stops.

You can learn more about this callback [here](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping).

While training, monitor the model's loss and accuracy on the 10,000 samples from the validation set.

In [ ]:
# The patience parameter is the amount of epochs to check for improvement.
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=200,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    callbacks=[early_stop])

## Evaluate the model

And let's see how the model performs. Two values will be returned. Loss (a number which represents our error, lower values are better), and accuracy.

In [ ]:
loss, accuracy = model.evaluate(test_data, test_labels)

print(loss, accuracy)

This fairly naive approach achieves an accuracy of about 87%. WIth more advanced approaches, the model should get closer to 95%.

## Create a graph of accuracy and loss over time

`model.fit()` returns a `History` object that contains a dictionary with everything that happened during training:

In [ ]:
history_dict = history.history
history_dict.keys()

There are four entries: one for each monitored metric during training and validation. We can use these to plot the training and validation loss for comparison, as well as the training and validation accuracy:

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = np.arange(len(acc)) + 1

# "bo" is for "blue dot"
_ = plt.plot(epochs, loss, 'bo', label='Training loss')

# b is for "solid blue line"
_ = plt.plot(epochs, val_loss, 'b', label='Validation loss')
_ = plt.title('Training and validation loss')
_ = plt.xlabel('Epochs')
_ = plt.ylabel('Loss')
_ = plt.legend()
plt.show();

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

_ = plt.plot(epochs, acc, 'bo', label='Training acc')
_ = plt.plot(epochs, val_acc, 'b', label='Validation acc')
_ = plt.title('Training and validation accuracy')
_ = plt.xlabel('Epochs')
_ = plt.ylabel('Accuracy')
_ = plt.legend()
plt.show();

Notice the training loss *decreases* with each epoch and the training accuracy *increases* with each epoch. This is expected when using a gradient descent optimization it should minimize the desired quantity on every iteration.

The validation accuracy peaks (and the validation loss bottoms) after about 40 epochs. After this point, the model over-optimizes and learns representations *specific* to the training data that do not *generalize* to test data.